In [2]:
!pip3 install --quiet openai python-dotenv dspy-ai


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [ ]:
!pip3 install --quiet arize-phoenix openinference-instrumentation-dspy opentelemetry-exporter-otlp

## Launch Arize Phoenix ##

In [11]:
import phoenix as px

px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## Set up Phoenix instrumentor ##

In [14]:
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

## Hello World ##

In [35]:
import dspy, os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
lm = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=4000)
dspy.settings.configure(lm=lm)

In [36]:
predictor = dspy.Predict("question -> answer")
print(predictor(question="hello?"))

Prediction(
    answer='Hello! How can I assist you today?'
)


**What is happening?**
![hello](./tracing_hello.jpg)

In [41]:
print(predictor(question="what is the capital city of France?"))

Prediction(
    answer='Paris'
)


![france](./tracing_france.jpg)

**So DSPy can decide the more appropriate prompt based on the question**

## Let's move on to fancy stuff ##

Looking into the "example" folder in DSPy, I think the "BasicQA" would be a good starting point. And copied over below. 

However, I changed the "desc" from ""often between 1 and 5 words"" to "whatever" because ... I am curious about it

In [42]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="whatever") # used to be often between 1 and 5 words"
class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate = dspy.Predict(BasicQA)

    def forward(self,question):
        prediction = self.generate(question = question)
        return dspy.Prediction(answer = prediction.answer)

In [43]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'Question: what is the capital city of France??\nAnswer: Paris'

![whatever](./tracing_whatever.jpg)

So, the desc ("whatever") is being used along with the "answer" field in the prompt

**Now let's give it a proper description, like "the city name"?**

In [44]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="the city name")

In [45]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'the city name'

![the city name desc](./tracing_desc.jpg)

**Ok let's get back to the "good" one**

In [23]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 2 words")

In [24]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'Paris'

**But what if we don't specify the desc**

In [48]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField() # no desc

In [49]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'Question: what is the capital city of France??\nAnswer: Paris'

In [50]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="the answer") # being naive again

In [52]:
pred = qa_bot.forward("what is the capital city of France??")
pred.answer

'Question: what is the capital city of France??\nAnswer: Paris'

**enough of desc**

In [32]:
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 2 words, or N/A")

In [33]:
qa_bot = BasicQABot()
pred = qa_bot.forward("what is the capital city of foiadhfioah;fg;hdasof??")
pred.answer

'N/A'

**to be fair, above experiments only demonstrate the unpredicable behavior how LLM react to our prompt, it is more about LLM than DSPy**